<a href="https://colab.research.google.com/github/beaterjy/demo-word2vec-gensim/blob/master/demo_word2vec_gensim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import jieba
import multiprocessing
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
from gensim.models import KeyedVectors

In [23]:
data = pd.read_excel('/content/drive/My Drive/Colab Notebooks/training_data_v2_4.xlsx')
data_z = pd.read_excel('/content/drive/My Drive/Colab Notebooks/multi_Z.xlsx')
print(data.shape)
print(data_z.shape)
data_z['phrase'] = data_z['verb'].str.cat(data_z['noun_without_bracket'], sep=' ', na_rep='')


(1922, 8)
(129672, 6)


In [24]:
data.head()

,门类,门类编号,大类,词组,动词,名词,大类编号,子类编号
0,农、林、牧、渔业,A,农业,谷物种植,种植,谷物,1,11
1,农、林、牧、渔业,A,农业,稻谷种植,种植,稻谷,1,111
2,农、林、牧、渔业,A,农业,小麦种植,种植,小麦,1,112
3,农、林、牧、渔业,A,农业,玉米种植,种植,玉米,1,113
4,农、林、牧、渔业,A,农业,其他谷物种植,种植,其他谷物,1,119


In [25]:
data_z.head(20)

,Unnamed: 0,level,zch,verb,noun,noun_without_bracket,phrase
0,0,0,440704000009296,销售,食品,食品,销售 食品
1,1,0,440704000009296,销售,日用杂品。,日用杂品,销售 日用杂品
2,2,1,440782000021799,自产自销,不锈钢,不锈钢,自产自销 不锈钢
3,3,1,440782000021799,自产自销,铝,铝,自产自销 铝
4,4,1,440782000021799,自产自销,铁,铁,自产自销 铁
5,5,1,440782000021799,自产自销,木制品,木制品,自产自销 木制品
6,6,1,440782000021799,自产自销,家用电器,家用电器,自产自销 家用电器
7,7,1,440782000021799,自产自销,包装材料,包装材料,自产自销 包装材料
8,8,1,440782000021799,自产自销,电子产品,电子产品,自产自销 电子产品
9,9,1,440782000021799,来料加工,不锈钢,不锈钢,来料加工 不锈钢


In [26]:
#停用词
def get_custom_stopwords(stop_words_file):
    with open(stop_words_file, encoding='gb18030') as f:
        stopwords = f.read()
    stopwords_list = stopwords.split('\n')
    custom_stopwords_list = [i for i in stopwords_list] #todo: ?
    return custom_stopwords_list

stop_words_file = '/content/drive/My Drive/Colab Notebooks/stopwordsHIT.txt'
stopwords = get_custom_stopwords(stop_words_file)
stopwords_set = frozenset(stopwords)
stopwords[:10]

['———', '》），', '）÷（１－', '”，', '）、', '＝（', ':', '→', '℃ ', '&']

In [0]:
#分词并处理停用词
def chinese_phrase_cut(text):
  text = text.replace(r'、', '')
  segs = jieba.cut(text)
  final = []
  for seg in segs:
    if seg not in stopwords_set:
      final.append(seg)
  return ' '.join(final)

data['phrase_cutted'] = data['词组'].apply(chinese_phrase_cut)
data_z['phrase_cutted'] = data_z['phrase'].apply(lambda x: chinese_phrase_cut(str(x)))

In [28]:
data.head(20)

,门类,门类编号,大类,词组,动词,名词,大类编号,子类编号,phrase_cutted
0,农、林、牧、渔业,A,农业,谷物种植,种植,谷物,1,11,谷物 种植
1,农、林、牧、渔业,A,农业,稻谷种植,种植,稻谷,1,111,稻谷 种植
2,农、林、牧、渔业,A,农业,小麦种植,种植,小麦,1,112,小麦 种植
3,农、林、牧、渔业,A,农业,玉米种植,种植,玉米,1,113,玉米 种植
4,农、林、牧、渔业,A,农业,其他谷物种植,种植,其他谷物,1,119,谷物 种植
5,农、林、牧、渔业,A,农业,豆类、油料和薯类种植,种植,豆类,1,12,豆类 油料 薯类 种植
6,农、林、牧、渔业,A,农业,豆类种植,种植,豆类,1,121,豆类 种植
7,农、林、牧、渔业,A,农业,油料种植,种植,油料,1,122,油料 种植
8,农、林、牧、渔业,A,农业,薯类种植,种植,薯类,1,123,薯类 种植
9,农、林、牧、渔业,A,农业,棉、麻、糖、烟草种植,种植,棉、麻、糖、烟草,1,13,棉 麻糖 烟草 种植


In [0]:
#构建X
A = data[['phrase_cutted']]
B = data_z[['phrase_cutted']]
X = pd.concat([A, B], axis=0)
X = X.reset_index(drop=True)
#标签y
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(data['大类'])
data['label'] = y

In [0]:
#保存txt
X.to_csv('/content/drive/My Drive/Colab Notebooks/phrase_cutted.txt', header=False, index=False)

In [31]:
X.head()

,phrase_cutted
0,谷物 种植
1,稻谷 种植
2,小麦 种植
3,玉米 种植
4,谷物 种植


In [32]:
#训练word2vec
dim = 300
def train_word2vec():
  inp = '/content/drive/My Drive/Colab Notebooks/phrase_cutted.txt'
  model = Word2Vec(LineSentence(inp), size=dim, window=1, min_count=5, )
  model.save('/content/drive/My Drive/Colab Notebooks/ph_cutted.model')
  model.wv.save_word2vec_format('/content/drive/My Drive/Colab Notebooks/ph_cutted.vector', binary=False)
train_word2vec()

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [33]:
#测试word2vec
# model = Word2Vec.load('/content/drive/My Drive/Colab Notebooks/ph_cutted.model')
# model = Word2Vec.load('/content/drive/My Drive/Colab Notebooks/baike_26g_news_13g_novel_229g.model')
model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/Colab Notebooks/zh.vec')
testwords = ['种植', '零售', '销售' ]
for x in testwords:
  res = model.most_similar(x)
  print(x)
  print(res)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


种植
[('经济作物', 0.7213346362113953), ('種植', 0.6631179451942444), ('农作物', 0.6582870483398438), ('小麦', 0.6486478447914124), ('品种', 0.6340972185134888), ('饲养', 0.6260604858398438), ('农业', 0.6255702972412109), ('水稻', 0.6255097389221191), ('耕种', 0.6231157779693604), ('茶叶', 0.6218434572219849)]
零售
[('零售商', 0.6607769727706909), ('零售店', 0.630832314491272), ('批發', 0.5688040256500244), ('批发', 0.5560111999511719), ('業務', 0.5524023175239563), ('商店', 0.5425912737846375), ('食肆', 0.5423717498779297), ('便利店', 0.5421520471572876), ('销售', 0.540289044380188), ('金融', 0.5396924614906311)]
销售
[('产品', 0.7279743552207947), ('销售额', 0.6790580153465271), ('生产', 0.6693570613861084), ('市场', 0.6567418575286865), ('销量', 0.6485793590545654), ('消费者', 0.6430990695953369), ('采购', 0.6395672559738159), ('生产商', 0.6333770751953125), ('进口', 0.628158450126648), ('企业', 0.6259716749191284)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
#合并关键词向量
def merge_vec(lst, dim):
  vec = np.zeros(dim)
  if not lst:
    return vec
  count = 0
  for w in lst:
    try:
      vec += model[w]
      count += 1
    except KeyError:
      continue
  vec /= count
  return vec

X_vec = X['phrase_cutted'].apply(lambda x: merge_vec(x.split(), dim))
X_vec = pd.DataFrame(X_vec.values.tolist())
X_vec = X_vec.replace(np.nan, 0)

In [0]:
#正则化
from sklearn.preprocessing import MinMaxScaler
X_vec = pd.DataFrame(MinMaxScaler().fit_transform(X_vec))

In [36]:
#划分训练集和测试集
from sklearn.model_selection import train_test_split
X_t = X_vec.head(len(A))
Z = X_vec.tail(len(B))
X_train, X_test, y_train, y_test = train_test_split(X_t, y, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(Z.shape)

(1441, 300)
(481, 300)
(129672, 300)


In [0]:
#贝叶斯模型
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
#决策树
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=1 )
#随机森林
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=500, criterion='gini', random_state=1, )
#SVM.LinearSVC
from sklearn.svm import LinearSVC
svc = LinearSVC()
#KNN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
#XGboost
from sklearn.ensemble import GradientBoostingClassifier
xgb = GradientBoostingClassifier()

In [38]:
#训练分数
from sklearn import metrics
m = svc
m = m.fit(X_train, y_train)
y_pred = m.predict(X_test)
#训练集分数
print(m.score(X_train, y_train))
#测试准确率
print(metrics.accuracy_score(y_test, y_pred))

0.8417765440666204
0.6340956340956341


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [39]:
#新数据预测结果
Z_pred = m.predict(Z)
data_z['predit'] = encoder.inverse_transform(Z_pred)
data_z.head(50)
# data_z.to_excel('/content/drive/My Drive/Colab Notebooks/Z_pre_v2.xlsx')

,Unnamed: 0,level,zch,verb,noun,noun_without_bracket,phrase,phrase_cutted,predit
0,0,0,440704000009296,销售,食品,食品,销售 食品,销售 食品,零售业
1,1,0,440704000009296,销售,日用杂品。,日用杂品,销售 日用杂品,销售 日用 杂品,零售业
2,2,1,440782000021799,自产自销,不锈钢,不锈钢,自产自销 不锈钢,自产自销 不锈钢,非金属矿物制品业
3,3,1,440782000021799,自产自销,铝,铝,自产自销 铝,自产自销 铝,通用设备制造业
4,4,1,440782000021799,自产自销,铁,铁,自产自销 铁,自产自销 铁,非金属矿物制品业
5,5,1,440782000021799,自产自销,木制品,木制品,自产自销 木制品,自产自销 木制品,公共设施管理业
6,6,1,440782000021799,自产自销,家用电器,家用电器,自产自销 家用电器,自产自销 家用电器,公共设施管理业
7,7,1,440782000021799,自产自销,包装材料,包装材料,自产自销 包装材料,自产自销 包装材料,公共设施管理业
8,8,1,440782000021799,自产自销,电子产品,电子产品,自产自销 电子产品,自产自销 电子产品,计算机、通信和其他电子设备制造业
9,9,1,440782000021799,来料加工,不锈钢,不锈钢,来料加工 不锈钢,来料加工 不锈钢,非金属矿物制品业
